In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

In [3]:
class Dataloader:
    def __init__(self, filepath):
        self.filepath = filepath
    
    def load_data(self):  
        # Error check for file existence
        try:
            self.df = pd.read_csv(self.filepath)
            print(f"✅ Data loaded successfully from {self.filepath}.")
            return self.df
        except FileNotFoundError:
            raise FileNotFoundError(f"Error: File not found at {self.filepath}")
        except Exception as e:
            raise Exception(f"An error occurred during data loading: {e}")

In [4]:
class DataAnalyzer:
    def __init__(self, dataframe):
        self.df = dataframe
    
    def show(self, choice, n=5):
        if choice == 'head':
            return self.df.head(n)
        elif choice == 'tail':
            return self.df.tail(n)
        elif choice == 'info':
            # df.info() prints directly, so we call it and return None
            self.df.info()
            return None
        elif choice == 'describe':
            return self.df.describe()
        elif choice == 'columns':
            return self.df.columns.tolist()
        elif choice == 'shape':
            return self.df.shape
        elif choice == 'dtypes':
            return self.df.dtypes
        elif choice == 'nulls':
            return self.df.isnull().sum()
        elif choice == 'duplicates':
            return self.df.duplicated().sum()
        else:
            raise ValueError("Invalid choice. Choose from: 'head', 'tail', 'info', 'describe', 'columns', 'shape', 'dtypes', 'nulls', 'duplicates'.")

In [5]:
class Preprocessor:
    def __init__(self, dataframe):
        if not isinstance(dataframe, pd.DataFrame):
            raise TypeError("dataframe must be a pandas DataFrame")
        self.df = dataframe.copy()

    def handle_missing_values(self, strategy: str = 'drop', fill_value=None, columns=None, inplace: bool = False):
        # ... (Your existing handle_missing_values code) ...
        allowed = {'drop', 'fill', 'ffill', 'bfill', 'mean', 'median', 'mode'}
        if not isinstance(strategy, str):
            raise TypeError("strategy must be a string")
        st = strategy.lower()
        if st not in allowed:
            raise ValueError(f"strategy must be one of {allowed}")

        columns = list(columns) if columns is not None else list(self.df.columns)
        missing = [c for c in columns if c not in self.df.columns]
        if missing:
            raise KeyError(f"columns not found in dataframe: {missing}")

        df = self.df if inplace else self.df.copy()

        if st == 'drop':
            df = df.dropna(subset=columns)

        elif st == 'fill':
            if isinstance(fill_value, dict):
                df = df.fillna(value={k: v for k, v in fill_value.items() if k in columns})
            else:
                df[columns] = df[columns].fillna(fill_value)

        elif st in ('ffill', 'bfill'):
            # Only apply ffill/bfill to columns that support it
            for c in columns:
                if not pd.api.types.is_numeric_dtype(df[c]) and not pd.api.types.is_categorical_dtype(df[c]):
                    print(f"Warning: Skipping ffill/bfill on non-numeric/non-categorical column '{c}'")
                    continue
                df[c] = df[c].fillna(method=st)

        elif st == 'mean':
            for c in columns:
                if pd.api.types.is_numeric_dtype(df[c]):
                    df[c].fillna(df[c].mean(), inplace=True)

        elif st == 'median':
            for c in columns:
                if pd.api.types.is_numeric_dtype(df[c]):
                    df[c].fillna(df[c].median(), inplace=True)

        elif st == 'mode':
            for c in columns:
                m = df[c].mode()
                if not m.empty:
                    df[c].fillna(m.iloc[0], inplace=True)
                
        if inplace:
            self.df = df
            return self.df
        return df

    def remove_columns(self, columns, inplace: bool = False):
        # ... (Your existing remove_columns code) ...
        if isinstance(columns, str):
            cols = [columns]
        else:
            cols = list(columns)
        missing = [c for c in cols if c not in self.df.columns]
        if missing:
            raise KeyError(f"columns not found in dataframe: {missing}")
        df = self.df if inplace else self.df.copy()
        df = df.drop(columns=cols)
        if inplace:
            self.df = df
            return self.df
        return df

    def remove_duplicates(self,inplace: bool = False):
        # ... (Your existing remove_duplicates code) ...
        df = self.df if inplace else self.df.copy()
        df = df.drop_duplicates()
        if inplace:
            self.df = df
            return self.df
        return df

    def encode_categorical(self, target_column=None, inplace: bool = False):
        """Encodes categorical columns (excluding the target if specified) using One-Hot and Label Encoding."""
        df = self.df if inplace else self.df.copy()
        
        # Identify categorical columns (excluding target)
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
        if target_column in categorical_cols:
            categorical_cols.remove(target_column)

        # 1. One-Hot Encode (for features)
        df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
        print(f"   -> One-Hot encoded features: {categorical_cols}")

        # 2. Label Encode Target (if applicable)
        if target_column and df[target_column].dtype in ['object', 'category']:
            le = LabelEncoder()
            df[target_column] = le.fit_transform(df[target_column])
            print(f"   -> Label encoded target column: {target_column}")

        if inplace:
            self.df = df
            return self.df
        return df
    
    def scale_numeric(self, columns=None, inplace: bool = False):
        """Scales numeric features using StandardScaler."""
        df = self.df if inplace else self.df.copy()
        
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Filter for specified columns if provided, otherwise use all numeric
        if columns is not None:
            numeric_cols = [col for col in numeric_cols if col in columns]

        if not numeric_cols:
            print("Warning: No numeric columns found or specified for scaling.")
            return df
            
        scaler = StandardScaler()
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
        print(f"   -> Scaled numeric features: {numeric_cols}")

        if inplace:
            self.df = df
            return self.df
        return df

In [7]:
class Visualizer:
    # ... (Your existing Visualizer code) ...
    def __init__(self, dataframe):
        if not isinstance(dataframe, pd.DataFrame):
            raise TypeError("dataframe must be a pandas DataFrame")
        self.df = dataframe.copy()

    def show_univariate_graph(self, column, plot_type,
                              x_label=None, y_label=None,
                              title=None, color=None,
                              max_categories=30, **kwargs):
        if column not in self.df.columns:
            print(f"Error: Column '{column}' not found in data.")
            return None

        data = self.df[column].dropna()
        if data.empty:
            print(f"Column '{column}' has no non-missing values to plot.")
            return None

        fig, ax = plt.subplots(figsize=(9, 6))

        try:
            if plot_type == 'histogram':
                sns.histplot(data=data, color=color, ax=ax, **kwargs)
                ax.set_ylabel(y_label or "Frequency / Density")

            elif plot_type == 'boxplot':
                sns.boxplot(y=data, color=color, ax=ax, **kwargs)
                ax.set_ylabel(y_label or "")

            elif plot_type == 'bar':
                if self.df[column].nunique() > max_categories:
                    print(f"Column '{column}' has >{max_categories} categories; showing top {max_categories}.")
                    top = self.df[column].value_counts().nlargest(max_categories).index
                    sns.countplot(data=self.df[self.df[column].isin(top)], x=column, color=color, ax=ax, **kwargs)
                else:
                    sns.countplot(data=self.df, x=column, color=color, ax=ax, **kwargs)
                ax.set_ylabel(y_label or "Count")

            else:
                print(f"Error: Plot type '{plot_type}' not supported. Choose from: ['histogram','boxplot','bar']")
                plt.close(fig)
                return None

            if x_label:
                ax.set_xlabel(x_label)
            if title:
                ax.set_title(title)
            else:
                ax.set_title(f"{plot_type.capitalize()} of {column}")

            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
            return ax

        except Exception as e:
            plt.close(fig)
            print(f"An error occurred during plotting: {type(e).__name__}: {e}")
            return None

    def show_bivariate_graph(self, x_column, y_column, plot_type,
                             x_label=None, y_label=None, title=None,
                             color=None, hue=None, max_points=1000, **kwargs):
        
        if x_column not in self.df.columns or y_column not in self.df.columns:
            print(f"Error: Columns '{x_column}' and/or '{y_column}' not found in data.")
            return None

        cols = [x_column, y_column]
        if hue and hue in self.df.columns:
            cols.append(hue)
        elif hue and hue not in self.df.columns:
             print(f"Warning: Hue column '{hue}' not found and will be ignored.")
             hue = None


        df_clean = self.df[cols].dropna()
        if df_clean.empty:
            print(f"No non-missing pairs to plot for '{x_column}' vs '{y_column}'.")
            return None

        if plot_type == 'scatter' and df_clean.shape[0] > max_points:
            df_plot = df_clean.sample(max_points, random_state=42)
        else:
            df_plot = df_clean

        fig, ax = plt.subplots(figsize=(9, 6))
        try:
            if plot_type == 'scatter':
                sns.scatterplot(data=df_plot, x=x_column, y=y_column, hue=hue, ax=ax, color=color, **kwargs)

            elif plot_type == 'hex':
                sns.histplot(data=df_plot, x=x_column, y=y_column, bins=30, cmap='viridis', cbar=True, ax=ax, **kwargs)

            elif plot_type == 'reg':
                sns.regplot(data=df_plot, x=x_column, y=y_column, scatter_kws={'s': 20}, line_kws={'color': 'red'}, ax=ax, **kwargs)

            elif plot_type == 'box':
                sns.boxplot(data=df_plot, x=x_column, y=y_column, hue=hue, palette=None if color is None else [color], ax=ax, **kwargs)

            elif plot_type == 'violin':
                sns.violinplot(data=df_plot, x=x_column, y=y_column, hue=hue, palette=None if color is None else [color], ax=ax, **kwargs)

            else:
                print(f"Error: Plot type '{plot_type}' not supported. Choose from: ['scatter','hex','reg','box','violin']")
                plt.close(fig)
                return None

            if x_label:
                ax.set_xlabel(x_label)
            if y_label:
                ax.set_ylabel(y_label)
            if title:
                ax.set_title(title)
            else:
                ax.set_title(f"{plot_type.capitalize()} of {y_column} vs {x_column}")

            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
            return ax

        except Exception as e:
            plt.close(fig)
            print(f"An error occurred during bivariate plotting: {type(e).__name__}: {e}")
            return None

In [8]:
class DataPreparer:
    # ... (Your existing DataPreparer code) ...
    def __init__(self, X, y, test_size=0.3, random_state=42):
        self.X = X
        self.y = y
        self.test_size = test_size
        self.random_state = random_state
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None

    def split_data(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=self.test_size, random_state=self.random_state
        )
        print("✅ Data split successfully.")
        return self.X_train, self.X_test, self.y_train, self.y_test

In [9]:
class ModelTrainer:
    # ... (Your existing ModelTrainer code) ...
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.models = {
            "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42, max_iter=1000), # Increased max_iter for safety
            "Decision Tree": DecisionTreeClassifier(random_state=42),
            "Support Vector Machine": SVC(random_state=42, probability=True)
        }
        self.trained_models = {}

    def train_models(self):
        print("\n🛠️ Starting model training...")
        for name, model in self.models.items():
            try:
                model.fit(self.X_train, self.y_train)
                self.trained_models[name] = model
                print(f"   -> {name} trained.")
            except Exception as e:
                print(f"   ❌ Error training {name}: {type(e).__name__} - {e}")
        print("✅ All models training attempted.")
        return self.trained_models

In [10]:
class ModelEvaluator:
    # ... (Your existing ModelEvaluator code) ...
    def __init__(self, trained_models, X_test, y_test):
        self.trained_models = trained_models
        self.X_test = X_test
        self.y_test = y_test
        self.results = {}

    def evaluate_models(self):
        print("\n📊 Starting model evaluation...")
        for name, model in self.trained_models.items():
            try:
                y_pred = model.predict(self.X_test)
                accuracy = accuracy_score(self.y_test, y_pred)
                report = classification_report(self.y_test, y_pred, output_dict=True, zero_division=0)
                
                self.results[name] = {
                    "accuracy": accuracy,
                    "y_pred": y_pred,
                    "report": report
                }
                print(f"   -> {name} evaluated. Accuracy: {accuracy:.4f}")
            except Exception as e:
                print(f"   ❌ Error evaluating {name}: {type(e).__name__} - {e}")
                
        print("✅ All models evaluated.")
        return self.results

    def find_best_model(self):
        best_model_name = ""
        highest_accuracy = -1
        
        # Error check: ensure results are not empty
        if not self.results:
            print("\n⚠️ No models were successfully evaluated.")
            return None, None
            
        for name, result in self.results.items():
            if result['accuracy'] > highest_accuracy:
                highest_accuracy = result['accuracy']
                best_model_name = name
                
        print("\n--- Model Performance Summary ---")
        for name, result in self.results.items():
             print(f"{name}: Accuracy = {result['accuracy']:.4f}")
             
        print("\n🏆 **Suggested Best Model:**")
        print(f"The **{best_model_name}** achieved the highest accuracy of **{highest_accuracy:.4f}**.")
        return best_model_name, highest_accuracy

In [11]:
def ensure_discrete_target(df, target_column):
    """Module-level helper to ensure a target column is discrete for classification.

    Same heuristics as before but available for users to call directly.
    Returns the modified DataFrame (or original if no changes applied).
    """
    if target_column not in df.columns:
        raise KeyError(f"Target column '{target_column}' not found in DataFrame.")

    y = df[target_column]
    print(f"-> Inspecting target '{target_column}': dtype={y.dtype}, uniques={y.nunique()}")

    # Case: non-numeric (strings/categories)
    if pd.api.types.is_object_dtype(y) or pd.api.types.is_categorical_dtype(y):
        print("-> Target appears categorical (object/category). Applying LabelEncoder.")
        le = LabelEncoder()
        df[target_column] = le.fit_transform(y.astype(str))
        return df

    # Case: numeric
    if pd.api.types.is_numeric_dtype(y):
        unique = y.dropna().unique()
        nunique = len(unique)

        # integer-like small number of uniques -> cast
        frac_max = 0
        if y.dropna().size:
            frac_max = (y.dropna() - y.dropna().round()).abs().max()
        if nunique <= 10 and frac_max == 0:
            print("-> Numeric target is integer-like with few unique values. Casting to integer labels.")
            df[target_column] = y.astype('Int64')
            return df

        # many values but many zeros (common for count targets) -> binarize
        zeros = (y == 0).sum() if y.size else 0
        if nunique > 10 and zeros > 0 and zeros / max(1, len(y)) > 0.2:
            print("-> Many unique numeric values but a large zero mass detected. Binarizing to (y>0).")
            df[target_column] = (y > 0).astype('Int64')
            return df

        # rounding reduces uniques to small set -> round and cast
        rounded = pd.Series(unique).round().unique()
        if len(rounded) <= 10:
            print("-> Rounding numeric target and casting to integer labels.")
            df[target_column] = y.round().astype('Int64')
            return df

    # fallback: leave as-is
    print("-> No safe automatic conversion applied to target; leave unchanged.")
    return df
       
      


In [ ]:
if __name__ == '__main__':
    # Example interactive workflow (no pipeline class). Create objects and call methods manually.
    FILE = "C:\\Users\\Computer\\OneDrive\\Desktop\\BSai projects\\heart.csv"
    TARGET = 'Target'

    # 1) Load
    dl = Dataloader(FILE)
    df = dl.load_data()

    # 2) Inspect
    analyzer = DataAnalyzer(df)
   
    print(analyzer.show('head', n=5))
    
    print(analyzer.show('describe'))

    # 3) Preprocess
    pre = Preprocessor(df)
    df = pre.remove_duplicates()
    df = pre.handle_missing_values(strategy='drop')
    # ensure target is discrete for classification
    df = ensure_discrete_target(df, TARGET)
    # encode categorical features (skips target)
    df = pre.encode_categorical(target_column=TARGET)
    # scale features (exclude target)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [c for c in numeric_cols if c != TARGET]
    df = pre.scale_numeric(columns=numeric_cols)

    # 4) Visualize (examples)
    viz = Visualizer(df)
    # show univariate
    _ = viz.show_univariate_graph(column=TARGET, plot_type='bar', title='Target distribution')
    # bivariate example: pick first numeric column vs target
    nums = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(nums) >= 1 and TARGET in df.columns:
        example_x = nums[0]
        _ = viz.show_bivariate_graph(x_column=example_x, y_column=TARGET, plot_type='scatter', title=f'{example_x} vs {TARGET}')

    # 5) Modeling (manual)
    X = df.drop(columns=[TARGET])
    y = df[TARGET]
    data_prep = DataPreparer(X, y)
    X_train, X_test, y_train, y_test = data_prep.split_data()
    trainer = ModelTrainer(X_train, y_train)
    models = trainer.train_models()
    evaluator = ModelEvaluator(models, X_test, y_test)
    results = evaluator.evaluate_models()
    best_model, best_acc = evaluator.find_best_model()
    if best_model:
        print(f"\nBest model: {best_model} (acc={best_acc:.4f})")